S31 (3L). Para los usuarios que tengan más de 100 reviews registradas, ¿cuál es el user_id cuya mediana del tiempo entre reviews es menor?

# Carga de archivos


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 64.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=933315d0d08e0dfbb9074e9e034332411afd4865ec4f89c8b910f472e6347e0d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
!apt install openjdk-8-jdk-headless -qq

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155629 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
from datetime import datetime as dt
import time

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1Unn0g_Kw2aEpvrmqJGBrrki9_YzWsEJG"})   # replace the id with id of file you want to access
downloaded.GetContentFile('review.csv') 

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
dfReview = (spark.read\
      .option("multiline", "true")\
      .option("quote", '"')\
      .option("header", "true")\
      .option("escape", "\\")\
      .option("escape", '"')\
      .csv('review.csv'))\
      .select(['user_id', 'date'])\
      .dropna()
rddReview = dfReview.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


primero filtro para los usuarios +100 reviews

# Calculo

In [ ]:
# pequeña funcion que devuelve el timestamp del formato del csv
def timestamp(string):
    return time.mktime(time.strptime(string, '%Y-%m-%d %H:%M:%S'))

Lo primero que hago es mapear un RDD del estilo (user_id, [ts]), con un reduceByKey sumo las listas y por ultimo filtro los user id que no cumplen que el largo de la lista es mayor a 100 (100 reviews) 

In [ ]:
most100 = rddReview.map(lambda x: (x[0],[ timestamp(x[1]) ]))\
                   .reduceByKey(lambda x,y : x + y)\
                   .filter(lambda x: (len(x[1]) > 100))


Aca lo que hago es ordenar cada timestamp de cada user_id para ordenarlos cronologicamente, luego hago otro map para con un for calcular las distancias entre todas las reviews (1ra-2da, 2da-3ra, 3ra-4ta .....) , y una vez que tengo esto lo reordeno

In [ ]:
distancesSorted = most100.map(lambda x: (x[0], sorted(x[1])))\
                         .map(lambda x: (x[0], [x[1][i+1] - x[1][i] for i in range(len(x[1]) - 1)]))\
                         .map(lambda x: (x[0], sorted(x[1])))

Como tengo las distancias entre reviews ordenadas, con acceder al elemento n/2 de la lista estoy accediendo a la mediana, asi que hago un map para obtener ese numero, por ultimo, con un reduce obtengo el user_id

In [ ]:
medianMin = distancesSorted.map(lambda x: (x[0], x[1][int(len(x[1])/ 2)]))\
                           .reduce(lambda x,y: x if x[1]<y[1] else y)
medianMin

('bQCHF5rn5lMI9c5kEwCaNA', 26.0)